# DenseNet121 (Transfer Learning)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from ptflops import get_model_complexity_info
from torchsummary import summary
import pandas as pd
import time

# Import custom utility module
from utils import get_transform, prepare_train_test_data, calculate_evaluation_metrics

In [3]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Load and prepare the data

In [4]:
# Transform data
image_input_size = (3, 256, 256) # Image input size for the model
transform = get_transform(resize_image_size = (256, 256))

# Prepare Train/Val/Test Data
train_loader, val_loader, test_loader = prepare_train_test_data(transform)

# Model keyword
model_keyword = "densenet121"

### Load pre-trained DenseNet model and modify it

In [5]:
# Load DenseNet121 model
model = models.densenet121(pretrained=True)

# Modify the classifier for binary classification
num_features = model.classifier.in_features
model.classifier = nn.Linear(num_features, 2)  # Binary classification: Internal waves (1) or No waves (0)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.1) # Implement a learning rate scheduler to adjust the learning rate during training

# Print the summary for DenseNet121
summary(model, input_size=image_input_size, device=str(device))

C:\Users\ishar\.virtualenvs\ML_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ishar\.virtualenvs\ML_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AttributeError: 'list' object has no attribute 'size'

### Train the model

In [29]:
# Initialize a DataFrame to log training/validation metrics
log_df = []

# Record the start time
start_time = time.time()

# Training loop
num_epochs = 50
best_val_loss = float('inf')  # Initialize best validation loss for saving the model

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = val_correct / val_total * 100
    
    # Adjust learning rate
    scheduler.step(val_loss)
    
    # Calculate FLOPs after each epoch
    with torch.no_grad():
        flops, _ = get_model_complexity_info(model, image_input_size, as_strings=False, print_per_layer_stat=False)

    # Log metrics into the DataFrame
    new_row = {
        "Epoch": epoch + 1,
        "Train_Loss": train_loss / len(train_loader),
        "Validation_Loss": val_loss / len(val_loader),
        "Validation_Accuracy": val_accuracy,
        "FLOPs": flops
    }
    log_df.append(new_row)

    # Print metrics for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss / len(train_loader):.4f}, "
          f"Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%, "
          f"GFLOPs: {(flops / 1e9 ):.2f}")

    # Save the best model based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "model_outputs_data/best_densenet121_model.pth")

# Calculate the total training time in seconds
print(f"\nTotal Training Time: {(time.time() - start_time):.2f} seconds")

# Save the DataFrame to a CSV file for later use
log_df = pd.DataFrame(log_df)
log_df.to_csv("model_outputs_data/model_evaluation_logs/training_logs_densenet121.csv", index=False)

Epoch [1/50], Train Loss: 0.4666, Val Loss: 0.4803, Val Accuracy: 81.20%, GFLOPs: 3.79
Epoch [2/50], Train Loss: 0.3745, Val Loss: 0.4637, Val Accuracy: 82.23%, GFLOPs: 3.79
Epoch [3/50], Train Loss: 0.3333, Val Loss: 0.3569, Val Accuracy: 84.43%, GFLOPs: 3.79
Epoch [4/50], Train Loss: 0.2823, Val Loss: 0.2738, Val Accuracy: 88.55%, GFLOPs: 3.79
Epoch [5/50], Train Loss: 0.2140, Val Loss: 0.3419, Val Accuracy: 87.52%, GFLOPs: 3.79
Epoch [6/50], Train Loss: 0.2377, Val Loss: 0.2293, Val Accuracy: 92.80%, GFLOPs: 3.79
Epoch [7/50], Train Loss: 0.2097, Val Loss: 0.3397, Val Accuracy: 87.67%, GFLOPs: 3.79
Epoch [8/50], Train Loss: 0.1587, Val Loss: 0.2535, Val Accuracy: 91.48%, GFLOPs: 3.79
Epoch [9/50], Train Loss: 0.1774, Val Loss: 0.3036, Val Accuracy: 88.11%, GFLOPs: 3.79
Epoch [10/50], Train Loss: 0.1504, Val Loss: 0.2426, Val Accuracy: 91.48%, GFLOPs: 3.79
Epoch [11/50], Train Loss: 0.1112, Val Loss: 0.2438, Val Accuracy: 90.75%, GFLOPs: 3.79
Epoch [12/50], Train Loss: 0.0826, Val Lo

### Prediction and Model evaluation

In [30]:
model.eval()
# Load the best model for testing
# model.load_state_dict(torch.load("model_outputs_data/best_densenet121_model.pth"))

# Test the model
all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Print evaluation metrics
calculate_evaluation_metrics(all_labels, all_preds, model_keyword)


Test Accuracy: 93.63%
Precision: 0.95
Recall: 0.92
F1-Score: 0.94

Classification Report:
                   precision    recall  f1-score   support

No Internal Waves       0.92      0.95      0.94       719
   Internal Waves       0.95      0.92      0.94       740

         accuracy                           0.94      1459
        macro avg       0.94      0.94      0.94      1459
     weighted avg       0.94      0.94      0.94      1459


Confusion Matrix:
[[685  34]
 [ 59 681]]
Predictions saved to model_outputs_data/model_prediction_logs/densenet121_labels_predictions.csv
